# Análisis Exploratorio de Datos (EDA) - Encuesta SERVQUAL Fundación Teletón

Este notebook realiza un análisis exploratorio completo de los datos de satisfacción de benefactores de Fundación Teletón, utilizando el marco teórico del modelo **SERVQUAL**.

## Marco Teórico: Modelo SERVQUAL

El modelo SERVQUAL (Parasuraman, Zeithaml y Berry, 1985) mide la calidad del servicio a través de 5 dimensiones:

| Dimensión | Descripción | Variables en este estudio |
|-----------|-------------|---------------------------|
| **Tangibles** | Apariencia física, equipos, personal | AT_1, AT_2 |
| **Fiabilidad** | Capacidad de realizar el servicio prometido | FI_1, FI_2, FI_3 |
| **Responsiveness** | Disposición y voluntad de ayudar | R_1, R_2, R_3 |
| **Seguridad/Garantía** | Conocimiento y cortesía del personal | (integrado en Fiabilidad) |
| **Empatía** | Atención individualizada al cliente | E_1, E_2, E_3, E_4 |

**Fuentes:**
- [SERVQUAL - Wikipedia](https://en.wikipedia.org/wiki/SERVQUAL)
- [SERVQUAL en Healthcare - PMC](https://pmc.ncbi.nlm.nih.gov/articles/PMC8535625/)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11
plt.rcParams['axes.titlesize'] = 14
plt.rcParams['axes.labelsize'] = 12

# Paleta de colores corporativa
COLORS = {
    'primary': '#1f77b4',
    'secondary': '#ff7f0e',
    'success': '#2ca02c',
    'danger': '#d62728',
    'warning': '#ffbb78',
    'info': '#17becf',
    'gray': '#7f7f7f'
}

# Colores para dimensiones SERVQUAL
SERVQUAL_COLORS = {
    'tangibles': '#3498db',
    'fiabilidad': '#e74c3c',
    'responsiveness': '#2ecc71',
    'empatia': '#9b59b6'
}

print("Librerías cargadas correctamente")

In [ ]:
# Cargar datos enriquecidos
df = pd.read_csv('../data/teleton_enriched.csv', encoding='utf-8-sig')

# Convertir fecha a datetime
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['fecha'] = pd.to_datetime(df['fecha'])

print(f"Dataset cargado: {df.shape[0]} registros x {df.shape[1]} columnas")
print(f"Período: {df['fecha'].min()} a {df['fecha'].max()}")

In [ ]:
# Definir grupos de variables para análisis
vars_servqual = ['AT_1', 'AT_2', 'FI_1', 'FI_2', 'FI_3', 'R_1', 'R_2', 'R_3', 'E_1', 'E_2', 'E_3', 'E_4']
vars_scores = ['score_tangibles', 'score_fiabilidad', 'score_responsiveness', 'score_empatia', 'score_servqual_total']
vars_outcome = ['D_1', 'NPS', 'C_1', 'INFO']
vars_categoricas = ['Giro', 'Puesto', 'Estado_limpio', 'region', 'nps_categoria', 'satisfaccion_nivel', 'calidad_nivel', 'antiguedad_grupo']

# Dimensiones SERVQUAL
dimensiones = {
    'tangibles': ['AT_1', 'AT_2'],
    'fiabilidad': ['FI_1', 'FI_2', 'FI_3'],
    'responsiveness': ['R_1', 'R_2', 'R_3'],
    'empatia': ['E_1', 'E_2', 'E_3', 'E_4']
}

---
# 1. Análisis Univariado

## 1.1 Estadísticas Descriptivas Generales

In [ ]:
# Estadísticas descriptivas de variables SERVQUAL
print("="*70)
print("ESTADÍSTICAS DESCRIPTIVAS - VARIABLES SERVQUAL (escala 1-5)")
print("="*70)
df[vars_servqual].describe().round(2)

In [ ]:
# Estadísticas de variables outcome
print("="*70)
print("ESTADÍSTICAS DESCRIPTIVAS - VARIABLES OUTCOME")
print("="*70)
df[vars_outcome + ['AÑOS']].describe().round(2)

### Interpretación Inicial

Las variables SERVQUAL muestran:
- Promedios entre 3.5 y 4.0 (escala 1-5), indicando una percepción **moderadamente positiva** del servicio
- Desviaciones estándar similares (~0.8), sugiriendo variabilidad consistente en las respuestas
- El percentil 25 está generalmente en 3, indicando que la mayoría de respuestas son neutrales o positivas

## 1.2 Distribución de Variables Numéricas

In [ ]:
# Distribución de ítems SERVQUAL
fig, axes = plt.subplots(3, 4, figsize=(16, 12))
axes = axes.flatten()

for i, var in enumerate(vars_servqual):
    ax = axes[i]
    counts = df[var].value_counts().sort_index()
    bars = ax.bar(counts.index, counts.values, color=COLORS['primary'], edgecolor='white')
    ax.set_title(f'{var}', fontweight='bold')
    ax.set_xlabel('')
    ax.set_ylabel('Frecuencia')
    ax.set_xticks([1, 2, 3, 4, 5])
    
    # Destacar la moda
    moda = counts.idxmax()
    for j, bar in enumerate(bars):
        if counts.index[j] == moda:
            bar.set_color(COLORS['secondary'])

plt.suptitle('Distribución de Ítems SERVQUAL (1=Totalmente en desacuerdo, 5=Totalmente de acuerdo)', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

### Hallazgo: Sesgo hacia valores positivos

La mayoría de las variables muestran una distribución sesgada hacia la derecha (valores 3, 4, 5), lo cual es común en encuestas de satisfacción. Esto indica una **percepción generalmente positiva** del servicio de los promotores de Teletón.

In [ ]:
# Distribución de variables outcome
fig, axes = plt.subplots(1, 4, figsize=(16, 4))

outcome_labels = {
    'D_1': 'Satisfacción (1-10)',
    'NPS': 'Recomendación NPS (1-10)',
    'C_1': 'Calidad Percibida (1-5)',
    'INFO': 'Nivel de Información (1-10)'
}

for i, var in enumerate(vars_outcome):
    ax = axes[i]
    ax.hist(df[var].dropna(), bins=10, color=COLORS['info'], edgecolor='white', alpha=0.8)
    ax.axvline(df[var].mean(), color=COLORS['danger'], linestyle='--', linewidth=2, label=f'Media: {df[var].mean():.1f}')
    ax.set_title(outcome_labels[var], fontweight='bold')
    ax.legend()

plt.suptitle('Distribución de Variables Outcome', fontsize=14, fontweight='bold', y=1.05)
plt.tight_layout()
plt.show()

In [ ]:
# Boxplots de scores SERVQUAL por dimensión
fig, ax = plt.subplots(figsize=(10, 6))

scores_data = df[vars_scores[:-1]].melt(var_name='Dimensión', value_name='Score')
scores_data['Dimensión'] = scores_data['Dimensión'].str.replace('score_', '').str.capitalize()

bp = ax.boxplot([df[f'score_{d}'].dropna() for d in dimensiones.keys()],
                labels=[d.capitalize() for d in dimensiones.keys()],
                patch_artist=True)

colors_list = list(SERVQUAL_COLORS.values())
for patch, color in zip(bp['boxes'], colors_list):
    patch.set_facecolor(color)
    patch.set_alpha(0.7)

ax.axhline(y=4, color='green', linestyle='--', alpha=0.5, label='Umbral positivo (4.0)')
ax.axhline(y=3, color='orange', linestyle='--', alpha=0.5, label='Punto neutro (3.0)')

ax.set_ylabel('Score (1-5)')
ax.set_title('Distribución de Scores por Dimensión SERVQUAL', fontweight='bold', fontsize=14)
ax.legend(loc='lower right')
ax.set_ylim(1, 5.5)

plt.tight_layout()
plt.show()

### Interpretación SERVQUAL por Dimensión

**Hallazgos clave:**
- **Empatía** y **Responsiveness** muestran medianas similares (~3.75)
- **Fiabilidad** presenta mayor variabilidad (caja más amplia)
- Todas las dimensiones tienen medianas por encima del punto neutro (3.0)
- Existen outliers inferiores en todas las dimensiones, indicando casos puntuales de insatisfacción

## 1.3 Distribución de Variables Categóricas

In [ ]:
# Distribución por Giro de empresa
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Giro
giro_counts = df['Giro'].value_counts()
ax1 = axes[0]
bars = ax1.barh(giro_counts.index, giro_counts.values, color=COLORS['primary'])
ax1.set_xlabel('Número de respuestas')
ax1.set_title('Distribución por Giro de Organización', fontweight='bold')
for i, v in enumerate(giro_counts.values):
    ax1.text(v + 1, i, f'{v} ({v/len(df)*100:.1f}%)', va='center')

# Puesto
puesto_counts = df['Puesto'].value_counts()
ax2 = axes[1]
bars = ax2.barh(puesto_counts.index, puesto_counts.values, color=COLORS['secondary'])
ax2.set_xlabel('Número de respuestas')
ax2.set_title('Distribución por Puesto del Encuestado', fontweight='bold')
for i, v in enumerate(puesto_counts.values):
    ax2.text(v + 1, i, f'{v} ({v/len(df)*100:.1f}%)', va='center')

plt.tight_layout()
plt.show()

### Perfil de los Encuestados

- **Educación** es el giro más representado, seguido de **Persona física** y **Empresa**
- Los puestos más comunes son **Persona física** y **Coordinador**
- Hay representación de diversos niveles jerárquicos (Director, Gerente, Dueño)

In [ ]:
# Distribución geográfica
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Por Estado (top 10)
estado_counts = df['Estado_limpio'].value_counts().head(10)
ax1 = axes[0]
bars = ax1.barh(estado_counts.index[::-1], estado_counts.values[::-1], color=COLORS['info'])
ax1.set_xlabel('Número de respuestas')
ax1.set_title('Top 10 Estados con más Respuestas', fontweight='bold')

# Por Región
region_counts = df['region'].value_counts()
ax2 = axes[1]
colors_region = plt.cm.Set3(np.linspace(0, 1, len(region_counts)))
ax2.pie(region_counts.values, labels=region_counts.index, autopct='%1.1f%%', 
        colors=colors_region, startangle=90)
ax2.set_title('Distribución por Región Geográfica', fontweight='bold')

plt.tight_layout()
plt.show()

### Cobertura Geográfica

- **Estado de México** concentra la mayor cantidad de respuestas
- La región **Centro** domina la muestra
- Hay representación de múltiples regiones del país, aunque desigual

In [ ]:
# Distribución de categorías NPS
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# NPS Categoría
nps_colors = {'Promotor': COLORS['success'], 'Pasivo': COLORS['warning'], 'Detractor': COLORS['danger']}
nps_counts = df['nps_categoria'].value_counts()
ax1 = axes[0]
bars = ax1.bar(nps_counts.index, nps_counts.values, color=[nps_colors.get(x, 'gray') for x in nps_counts.index])
ax1.set_title('Distribución NPS', fontweight='bold')
ax1.set_ylabel('Frecuencia')
for bar, val in zip(bars, nps_counts.values):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, f'{val}\n({val/len(df)*100:.1f}%)', 
             ha='center', fontweight='bold')

# Satisfacción Nivel
sat_colors = {'Alto': COLORS['success'], 'Medio': COLORS['warning'], 'Bajo': COLORS['danger']}
sat_counts = df['satisfaccion_nivel'].value_counts()
ax2 = axes[1]
bars = ax2.bar(sat_counts.index, sat_counts.values, color=[sat_colors.get(x, 'gray') for x in sat_counts.index])
ax2.set_title('Nivel de Satisfacción', fontweight='bold')
ax2.set_ylabel('Frecuencia')
for bar, val in zip(bars, sat_counts.values):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, f'{val}\n({val/len(df)*100:.1f}%)', 
             ha='center', fontweight='bold')

# Calidad Nivel
cal_colors = {'Bueno': COLORS['success'], 'Regular': COLORS['warning'], 'Deficiente': COLORS['danger']}
cal_counts = df['calidad_nivel'].value_counts()
ax3 = axes[2]
bars = ax3.bar(cal_counts.index, cal_counts.values, color=[cal_colors.get(x, 'gray') for x in cal_counts.index])
ax3.set_title('Calidad Percibida', fontweight='bold')
ax3.set_ylabel('Frecuencia')
for bar, val in zip(bars, cal_counts.values):
    ax3.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 2, f'{val}\n({val/len(df)*100:.1f}%)', 
             ha='center', fontweight='bold')

plt.suptitle('Distribución de Métricas de Experiencia', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Calcular NPS Score
nps_pct = df['nps_categoria'].value_counts(normalize=True) * 100
nps_score = nps_pct.get('Promotor', 0) - nps_pct.get('Detractor', 0)

print("="*50)
print("NPS SCORE DE FUNDACIÓN TELETÓN")
print("="*50)
print(f"\nNPS Score: {nps_score:.1f}")
print(f"\n  Promotores (9-10): {nps_pct.get('Promotor', 0):.1f}%")
print(f"  Pasivos (7-8):     {nps_pct.get('Pasivo', 0):.1f}%")
print(f"  Detractores (1-6): {nps_pct.get('Detractor', 0):.1f}%")
print("\n" + "="*50)
if nps_score > 50:
    print("Interpretación: EXCELENTE - NPS > 50")
elif nps_score > 0:
    print("Interpretación: BUENO - NPS positivo")
else:
    print("Interpretación: NECESITA MEJORA - NPS negativo")

---
# 2. Análisis por Dimensión SERVQUAL

In [ ]:
# Comparación de scores promedio por dimensión
scores_promedio = {}
for dim, items in dimensiones.items():
    scores_promedio[dim.capitalize()] = df[items].mean().mean()

fig, ax = plt.subplots(figsize=(10, 6))

dims = list(scores_promedio.keys())
vals = list(scores_promedio.values())
colors_dims = [SERVQUAL_COLORS[d.lower()] for d in dims]

bars = ax.barh(dims, vals, color=colors_dims, edgecolor='white', height=0.6)
ax.axvline(x=4, color='green', linestyle='--', alpha=0.7, label='Umbral positivo')
ax.axvline(x=3, color='orange', linestyle='--', alpha=0.7, label='Punto neutro')

for bar, val in zip(bars, vals):
    ax.text(val + 0.05, bar.get_y() + bar.get_height()/2, f'{val:.2f}', va='center', fontweight='bold', fontsize=12)

ax.set_xlim(0, 5)
ax.set_xlabel('Score Promedio (1-5)')
ax.set_title('Score Promedio por Dimensión SERVQUAL', fontweight='bold', fontsize=14)
ax.legend(loc='lower right')

plt.tight_layout()
plt.show()

### Interpretación SERVQUAL

**Fortalezas identificadas:**
- La dimensión con mayor score indica el área donde los promotores de Teletón destacan

**Áreas de oportunidad:**
- Las dimensiones por debajo del umbral de 4.0 representan oportunidades de mejora

In [ ]:
# Análisis detallado por ítem dentro de cada dimensión
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

item_labels = {
    'AT_1': 'Vestimenta/Comportamiento',
    'AT_2': 'Documentación clara',
    'FI_1': 'Cumple horarios',
    'FI_2': 'Conocimiento',
    'FI_3': 'Info correcta',
    'R_1': 'Respuesta rápida',
    'R_2': 'Disposición ayudar',
    'R_3': 'Flexibilidad',
    'E_1': 'Actitud comprensiva',
    'E_2': 'Dedica tiempo',
    'E_3': 'Se preocupa',
    'E_4': 'Atención personalizada'
}

for idx, (dim, items) in enumerate(dimensiones.items()):
    ax = axes.flatten()[idx]
    means = [df[item].mean() for item in items]
    labels = [item_labels[item] for item in items]
    
    bars = ax.barh(labels, means, color=SERVQUAL_COLORS[dim], alpha=0.8)
    ax.axvline(x=4, color='green', linestyle='--', alpha=0.5)
    ax.axvline(x=3, color='orange', linestyle='--', alpha=0.5)
    
    for bar, val in zip(bars, means):
        ax.text(val + 0.05, bar.get_y() + bar.get_height()/2, f'{val:.2f}', va='center', fontsize=10)
    
    ax.set_xlim(0, 5)
    ax.set_title(f'Dimensión: {dim.upper()}', fontweight='bold', color=SERVQUAL_COLORS[dim])
    ax.set_xlabel('Score promedio')

plt.suptitle('Análisis Detallado por Ítem SERVQUAL', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# Identificar ítems con mayor y menor score
item_scores = {item: df[item].mean() for item in vars_servqual}
item_scores_sorted = sorted(item_scores.items(), key=lambda x: x[1], reverse=True)

print("="*60)
print("RANKING DE ÍTEMS SERVQUAL (de mayor a menor score)")
print("="*60)
for i, (item, score) in enumerate(item_scores_sorted, 1):
    emoji = "✅" if score >= 4 else "⚠️" if score >= 3.5 else "❌"
    print(f"{i:2d}. {item}: {score:.2f} {emoji} - {item_labels[item]}")

---
# 3. Análisis Bivariado y Correlaciones

In [ ]:
# Matriz de correlación
vars_correlacion = vars_scores + vars_outcome + ['AÑOS']
corr_matrix = df[vars_correlacion].corr()

fig, ax = plt.subplots(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, fmt='.2f', cmap='RdYlBu_r', 
            center=0, vmin=-1, vmax=1, ax=ax, square=True,
            cbar_kws={'label': 'Correlación'})
ax.set_title('Matriz de Correlación: Scores SERVQUAL y Variables Outcome', fontweight='bold', fontsize=14)
plt.tight_layout()
plt.show()

### Hallazgos de Correlación

**Correlaciones más fuertes con NPS y Satisfacción (D_1):**
- Identificar qué dimensiones SERVQUAL tienen mayor impacto en la recomendación y satisfacción

In [ ]:
# Correlaciones con NPS
corr_nps = df[vars_scores[:-1]].corrwith(df['NPS']).sort_values(ascending=False)

print("="*50)
print("CORRELACIÓN DE DIMENSIONES SERVQUAL CON NPS")
print("="*50)
for dim, corr in corr_nps.items():
    strength = "Fuerte" if abs(corr) > 0.5 else "Moderada" if abs(corr) > 0.3 else "Débil"
    print(f"{dim.replace('score_', '').capitalize():15s}: r = {corr:.3f} ({strength})")

In [ ]:
# Scatter plots: Scores SERVQUAL vs NPS
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

for idx, dim in enumerate(dimensiones.keys()):
    ax = axes.flatten()[idx]
    score_col = f'score_{dim}'
    
    ax.scatter(df[score_col], df['NPS'], alpha=0.5, c=SERVQUAL_COLORS[dim], s=50)
    
    # Línea de tendencia
    z = np.polyfit(df[score_col].dropna(), df.loc[df[score_col].notna(), 'NPS'], 1)
    p = np.poly1d(z)
    x_line = np.linspace(df[score_col].min(), df[score_col].max(), 100)
    ax.plot(x_line, p(x_line), "--", color='red', linewidth=2, label=f'Tendencia')
    
    corr = df[score_col].corr(df['NPS'])
    ax.set_xlabel(f'Score {dim.capitalize()}')
    ax.set_ylabel('NPS')
    ax.set_title(f'{dim.capitalize()} vs NPS (r = {corr:.2f})', fontweight='bold')
    ax.legend()

plt.suptitle('Relación entre Dimensiones SERVQUAL y NPS', fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

---
# 4. Análisis por Segmentos

In [ ]:
# Satisfacción por Giro de empresa
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Satisfacción promedio por Giro
sat_por_giro = df.groupby('Giro')['D_1'].mean().sort_values(ascending=True)
ax1 = axes[0]
bars = ax1.barh(sat_por_giro.index, sat_por_giro.values, color=COLORS['primary'])
ax1.axvline(x=df['D_1'].mean(), color='red', linestyle='--', label=f'Promedio general: {df["D_1"].mean():.1f}')
ax1.set_xlabel('Satisfacción Promedio (1-10)')
ax1.set_title('Satisfacción por Tipo de Organización', fontweight='bold')
ax1.legend()
for bar, val in zip(bars, sat_por_giro.values):
    ax1.text(val + 0.1, bar.get_y() + bar.get_height()/2, f'{val:.1f}', va='center')

# NPS promedio por Giro
nps_por_giro = df.groupby('Giro')['NPS'].mean().sort_values(ascending=True)
ax2 = axes[1]
bars = ax2.barh(nps_por_giro.index, nps_por_giro.values, color=COLORS['secondary'])
ax2.axvline(x=df['NPS'].mean(), color='red', linestyle='--', label=f'Promedio general: {df["NPS"].mean():.1f}')
ax2.set_xlabel('NPS Promedio (1-10)')
ax2.set_title('NPS por Tipo de Organización', fontweight='bold')
ax2.legend()
for bar, val in zip(bars, nps_por_giro.values):
    ax2.text(val + 0.1, bar.get_y() + bar.get_height()/2, f'{val:.1f}', va='center')

plt.tight_layout()
plt.show()

In [ ]:
# Análisis por antigüedad
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Boxplot Satisfacción por antigüedad
orden_antiguedad = ['Nuevo', 'Establecido', 'Veterano']
ax1 = axes[0]
df.boxplot(column='D_1', by='antiguedad_grupo', ax=ax1, positions=[orden_antiguedad.index(x) for x in df['antiguedad_grupo'].dropna().unique()])
ax1.set_xlabel('Antigüedad')
ax1.set_ylabel('Satisfacción (1-10)')
ax1.set_title('Satisfacción por Antigüedad del Benefactor', fontweight='bold')
plt.suptitle('')

# Score SERVQUAL total por antigüedad
servqual_por_antiguedad = df.groupby('antiguedad_grupo')['score_servqual_total'].mean()
ax2 = axes[1]
bars = ax2.bar(orden_antiguedad, [servqual_por_antiguedad.get(x, 0) for x in orden_antiguedad], 
               color=[COLORS['success'], COLORS['warning'], COLORS['info']])
ax2.axhline(y=df['score_servqual_total'].mean(), color='red', linestyle='--', 
            label=f'Promedio: {df["score_servqual_total"].mean():.2f}')
ax2.set_xlabel('Antigüedad')
ax2.set_ylabel('Score SERVQUAL Total')
ax2.set_title('Percepción de Calidad por Antigüedad', fontweight='bold')
ax2.legend()
ax2.set_ylim(0, 5)

plt.tight_layout()
plt.show()

### Hallazgo: Efecto de la Antigüedad

Analizar si los benefactores más antiguos (veteranos) tienen diferente percepción que los nuevos.

In [ ]:
# Análisis geográfico - Satisfacción por región
fig, ax = plt.subplots(figsize=(12, 6))

region_stats = df.groupby('region').agg({
    'D_1': 'mean',
    'NPS': 'mean',
    'score_servqual_total': 'mean',
    'Estado_limpio': 'count'
}).rename(columns={'Estado_limpio': 'n_respuestas'}).sort_values('D_1', ascending=True)

x = np.arange(len(region_stats))
width = 0.25

bars1 = ax.barh(x - width, region_stats['D_1'], width, label='Satisfacción', color=COLORS['primary'])
bars2 = ax.barh(x, region_stats['NPS'], width, label='NPS', color=COLORS['secondary'])
bars3 = ax.barh(x + width, region_stats['score_servqual_total'] * 2, width, label='SERVQUAL (x2)', color=COLORS['success'])

ax.set_yticks(x)
ax.set_yticklabels(region_stats.index)
ax.set_xlabel('Score')
ax.set_title('Métricas de Satisfacción por Región Geográfica', fontweight='bold', fontsize=14)
ax.legend(loc='lower right')

# Agregar n de cada región
for i, (idx, row) in enumerate(region_stats.iterrows()):
    ax.text(10.5, i, f'n={int(row["n_respuestas"])}', va='center', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# Tabla resumen por región
print("="*80)
print("RESUMEN DE MÉTRICAS POR REGIÓN")
print("="*80)
print(region_stats.round(2).to_string())

---
# 5. Patrones Temporales

In [ ]:
# Respuestas por día
respuestas_diarias = df.groupby('fecha').size()

fig, ax = plt.subplots(figsize=(14, 5))
ax.plot(respuestas_diarias.index, respuestas_diarias.values, marker='o', linewidth=2, markersize=4)
ax.fill_between(respuestas_diarias.index, respuestas_diarias.values, alpha=0.3)
ax.set_xlabel('Fecha')
ax.set_ylabel('Número de respuestas')
ax.set_title('Evolución de Respuestas de la Encuesta', fontweight='bold', fontsize=14)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# Satisfacción por turno del día
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

turno_order = ['Mañana', 'Tarde', 'Noche']
turno_colors = ['#f1c40f', '#e67e22', '#2c3e50']

# Distribución de respuestas por turno
turno_counts = df['turno'].value_counts().reindex(turno_order)
ax1 = axes[0]
ax1.pie(turno_counts.values, labels=turno_counts.index, autopct='%1.1f%%', colors=turno_colors, startangle=90)
ax1.set_title('Distribución de Respuestas por Turno', fontweight='bold')

# Satisfacción promedio por turno
sat_por_turno = df.groupby('turno')['D_1'].mean().reindex(turno_order)
ax2 = axes[1]
bars = ax2.bar(turno_order, sat_por_turno.values, color=turno_colors)
ax2.axhline(y=df['D_1'].mean(), color='red', linestyle='--', label=f'Promedio: {df["D_1"].mean():.1f}')
ax2.set_ylabel('Satisfacción Promedio')
ax2.set_title('Satisfacción por Turno de Respuesta', fontweight='bold')
ax2.legend()

for bar, val in zip(bars, sat_por_turno.values):
    ax2.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1, f'{val:.1f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

---
# 6. Análisis de Gaps SERVQUAL

In [ ]:
# Calcular gap respecto al máximo esperado (5)
gaps = {}
for dim, items in dimensiones.items():
    score_actual = df[items].mean().mean()
    gap = 5 - score_actual  # Distancia al máximo
    gaps[dim.capitalize()] = {
        'score': score_actual,
        'gap': gap,
        'cumplimiento': (score_actual / 5) * 100
    }

gaps_df = pd.DataFrame(gaps).T

print("="*60)
print("ANÁLISIS DE GAPS SERVQUAL")
print("="*60)
print(f"\n{'Dimensión':<15} {'Score':>8} {'Gap':>8} {'Cumplimiento':>12}")
print("-"*45)
for dim, data in gaps.items():
    print(f"{dim:<15} {data['score']:>8.2f} {data['gap']:>8.2f} {data['cumplimiento']:>11.1f}%")

In [ ]:
# Gráfico de radar SERVQUAL
from math import pi

categories = list(dimensiones.keys())
N = len(categories)

# Valores
values = [df[f'score_{cat}'].mean() for cat in categories]
values += values[:1]  # Cerrar el círculo

# Ángulos
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

# Dibujar el gráfico
ax.plot(angles, values, 'o-', linewidth=2, color=COLORS['primary'], label='Score actual')
ax.fill(angles, values, alpha=0.25, color=COLORS['primary'])

# Línea de referencia (máximo)
max_values = [5] * (N + 1)
ax.plot(angles, max_values, '--', linewidth=1, color=COLORS['danger'], label='Máximo (5)')

# Línea de referencia (4 = positivo)
ref_values = [4] * (N + 1)
ax.plot(angles, ref_values, '--', linewidth=1, color=COLORS['success'], label='Objetivo (4)')

# Etiquetas
ax.set_xticks(angles[:-1])
ax.set_xticklabels([cat.capitalize() for cat in categories], size=12)
ax.set_ylim(0, 5)

ax.set_title('Perfil SERVQUAL de Fundación Teletón', fontweight='bold', fontsize=14, y=1.08)
ax.legend(loc='upper right', bbox_to_anchor=(1.3, 1))

plt.tight_layout()
plt.show()

### Interpretación del Perfil SERVQUAL

El gráfico de radar muestra:
- **Área azul**: Desempeño actual en cada dimensión
- **Línea roja**: Máximo posible (5)
- **Línea verde**: Objetivo deseable (4)

Las dimensiones que no alcanzan la línea verde representan las principales áreas de mejora.

---
# 7. Conclusiones del EDA

## Resumen Ejecutivo

In [ ]:
# Generar resumen ejecutivo automático
print("="*70)
print("RESUMEN EJECUTIVO - ANÁLISIS SERVQUAL FUNDACIÓN TELETÓN")
print("="*70)

# Métricas clave
print(f"\n📊 MÉTRICAS GENERALES")
print(f"   • Total de respuestas: {len(df)}")
print(f"   • Satisfacción promedio: {df['D_1'].mean():.1f}/10")
print(f"   • NPS Score: {nps_score:.1f}")
print(f"   • Calidad percibida: {df['C_1'].mean():.2f}/5")
print(f"   • Score SERVQUAL total: {df['score_servqual_total'].mean():.2f}/5")

# Dimensión más fuerte
dim_scores = {dim: df[f'score_{dim}'].mean() for dim in dimensiones.keys()}
dim_mejor = max(dim_scores, key=dim_scores.get)
dim_peor = min(dim_scores, key=dim_scores.get)

print(f"\n💪 FORTALEZAS (Dimensión más alta)")
print(f"   • {dim_mejor.upper()}: {dim_scores[dim_mejor]:.2f}/5")

print(f"\n⚠️ ÁREAS DE OPORTUNIDAD (Dimensión más baja)")
print(f"   • {dim_peor.upper()}: {dim_scores[dim_peor]:.2f}/5")

# Segmentos
print(f"\n👥 PERFIL DE BENEFACTORES")
giro_top = df['Giro'].value_counts().index[0]
estado_top = df['Estado_limpio'].value_counts().index[0]
print(f"   • Giro predominante: {giro_top}")
print(f"   • Estado con más respuestas: {estado_top}")
print(f"   • Antigüedad promedio: {df['AÑOS'].mean():.1f} años")

# Correlaciones clave
print(f"\n🔗 CORRELACIONES CLAVE CON NPS")
for dim in dimensiones.keys():
    corr = df[f'score_{dim}'].corr(df['NPS'])
    print(f"   • {dim.capitalize()}: r = {corr:.3f}")

## Conclusiones Principales

### 1. Percepción General del Servicio
Los benefactores de Fundación Teletón tienen una percepción **moderadamente positiva** del servicio de los promotores, con scores SERVQUAL cercanos a 4/5 en la mayoría de dimensiones.

### 2. NPS y Lealtad
El NPS Score indica el nivel de lealtad de los benefactores. La mayoría son pasivos (7-8), lo que representa una oportunidad para convertirlos en promotores activos.

### 3. Dimensiones SERVQUAL
- **Fortaleza principal**: La dimensión con mayor score refleja el área donde los promotores destacan más.
- **Área de mejora prioritaria**: La dimensión con menor score debe ser el foco de capacitación y mejora.

### 4. Segmentación
- Las instituciones educativas representan el segmento más grande
- Existe variabilidad geográfica en las percepciones de calidad
- La antigüedad del benefactor puede influir en su percepción

### 5. Recomendaciones
1. Fortalecer la capacitación en las dimensiones con menor score
2. Implementar programas de reconocimiento para promotores con alto desempeño en las dimensiones fuertes
3. Diseñar estrategias diferenciadas por tipo de organización y región
4. Establecer métricas de seguimiento basadas en las dimensiones SERVQUAL

In [ ]:
# Guardar resumen para dashboard
resumen = {
    'total_respuestas': len(df),
    'satisfaccion_promedio': df['D_1'].mean(),
    'nps_score': nps_score,
    'calidad_promedio': df['C_1'].mean(),
    'servqual_total': df['score_servqual_total'].mean(),
    'dimension_mejor': dim_mejor,
    'dimension_peor': dim_peor,
    'scores_dimensiones': dim_scores
}

print("\nResumen guardado para uso en dashboard.")
print(resumen)